In [3]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

ksdir = '/home/anverdie/Data_Ephys/793616_D5_16092022'

In [10]:
# Load digital input
f = open(os.path.join('/home/anverdie/Data_Ephys/793616_D5_16092022', 'digitalin.dat'), 'rb') 
sd_array = np.fromfile(f, np.uint16)
f.close()
sd_array_on = np.where(np.insert(np.diff(sd_array), 0, 0) == 4)[0]
sd_array_off = np.where(np.insert(np.diff(sd_array), 0, 0) == -4)[0]

# For some reason there is 26 uknown pulses at the beginning of the recording
sd_array_on = sd_array_on[26:]
sd_array_off = sd_array_off[26:]

In [17]:
sp_times = np.load(os.path.join(ksdir,'spike_times.npy'))
sp_times = np.array([i[0] for i in sp_times])

sp_clu = np.load(os.path.join(ksdir,'spike_clusters.npy'))

clu_gp = pd.read_csv(os.path.join(ksdir,'cluster_group.tsv'), delimiter='\t')
clu_gp = clu_gp['group'].to_numpy()

usable_clusters = [i for i, clu in enumerate(clu_gp) if clu == 'good']

usable_sp = np.isin(sp_clu, usable_clusters)
sp_times = sp_times[usable_sp]
sp_clu = sp_clu[usable_sp]

idxs_clu = []
for clu in np.unique(sp_clu):
    idxs_clu.append(np.where(sp_clu == clu)[0])
n_clu = len(idxs_clu)

In [46]:
stims_ttls = np.arange(0, len(sd_array_on), 10)
first_ttls = [sd_array_on[s] for s in stims_ttls]

In [100]:
# Structure of array is dict[stim_number][presentation_number] containg a number_of_cluster * spikes matrix
presentation = {}
for i, ttl in enumerate(first_ttls):
    spikes = sp_times[sp_times > ttl]
    spikes = spikes[spikes < ttl + 10000]

    presentation[i] = [(np.intersect1d(sp_times[idx_clu], spikes) - ttl)/20 for idx_clu in idxs_clu]


In [101]:
# Averageover all presentations
n_pres = 15
n_stim_pre_pres = 144
n_chan = 65

mean_pres = {}
for i in range(n_stim_pre_pres):
    mean_pres[i] = {}
    for chan in range(n_chan):
        mean_pres[i][chan] = np.array([])


for p in range(n_pres):
    for s in range(n_stim_pre_pres):
        for chan in range(len(presentation[0])):
            mean_pres[s][chan] = np.append(mean_pres[s][chan], presentation[p*144+s][chan])

In [105]:
for k in range(144):
    print(np.sum([len(mean_pres[k][c]) for c in range(n_chan)]))

2150
2227
2456
2477
2662
2154
2249
2141
2656
2541
2479
2045
2741
2813
2455
2870
2508
2272
2436
2623
2319
2133
2096
2131
1617
1369
1743
1209
2630
2688
2665
2372
2507
2835
2819
2530
2354
3010
2438
2746
2859
2937
2563
2721
2544
2689
2617
2609
2554
3089
3167
2773
2798
2635
2676
2652
2509
2540
2310
2527
3049
2735
2457
3044
2477
2878
2793
2651
2889
2748
2675
2790
2601
2698
2666
2692
2523
2733
2815
2655
2637
2416
2673
2875
2635
2795
2589
2371
2467
2889
2429
2973
2714
2550
2864
2682
2706
2748
2531
2881
3051
2965
2595
2781
2856
2521
2342
2884
2754
2559
2763
2790
2861
3186
3222
2795
2858
2897
2719
2776
2630
2700
2416
2850
2283
2914
2861
2616
2471
3178
2864
2940
2457
2445
2618
2653
2202
2408
2287
2990
2472
2923
3095
2811


In [106]:
int_5 = np.arange(0, 144, 4)
int_10 = np.arange(1, 144, 4)
int_15 = np.arange(2, 144, 4)
int_20 = np.arange(3, 144, 4)

In [127]:
intensities = [int_5, int_10, int_15, int_20]
name_protocol = ['5mW', '10mW', '15mW', '20mW']
for c in range(n_chan):
    #fig, axs = plt.subplots(1, 4)
    evo_spike = []
    for i, intensity in enumerate(intensities):
        heat = np.array([len(mean_pres[led][c]) for led in intensity]).reshape(6, 6)
        evo_spike.append(np.max([len(mean_pres[led][c])for led in intensity])/np.max([len(mean_pres[led][c])for led in int_5]))
    plt.plot(evo_spike)
    plt.savefig('implant_heatmaps/mean_channels_{}.png'.format(c))
    plt.close()

    #     axs[i].imshow(heat)
    #     axs[i].set_title('{}'.format(name_protocol[i]))
    # plt.savefig('implant_heatmaps/channel_{}.png'.format(c))
    # plt.close()
